In [1]:
from PIL import Image
from io import BytesIO
import os, sys, requests, json, random, math, imageio, shutil, glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage.filters import threshold_otsu
from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.measure import label, regionprops
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import cv2
# sys.path.append('/data/users/g6114450078/PyTorch-YOLOv3/preprocessing')
# from aug import *

In [2]:
# get images from pf_ext
# labels = os.path.join('Pf_ext_01.json')

count = 1
phase_count = {
    "train": 1,
    "test": 1
}

phase = "train"
# degs = [45,90,135,180,225,270,315]

# data = (requests.get('https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/Pf_ext_01.json')).json()
# images = data["_via_img_metadata"]
# for key in images:
#     url = images[key]["filename"]
#     leng = len(images[key]["regions"])
#     bboxs = ""
#     if leng > 0 and count <= 20:
#         name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
#         print("{} {}".format(count,url))
#         print(name)
#         response = requests.get(url)
#         img = Image.open(BytesIO(response.content))
#         w, h = img.size
#         img.save(os.path.join('../data/custom/images',name+".jpg"))
#         f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
#         rects = []
#         for box in images[key]["regions"]:
#             b = box["shape_attributes"]
#             x1 = b["x"]/w
#             y1 = b["y"]/h
#             bw = b["width"]/w
#             bh = b["height"]/h
#             xc = x1 + (bw/2)
#             yc = y1 + (bh/2)
#             lbl = box["region_attributes"]["type"]
#             if lbl == "2":
#                 lbl = "1"
#                 bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
#                 rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
#             elif lbl == "1":
#                 lbl = "0"
#                 bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
#                 rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
#         bboxs = bboxs[1:]
#         f.write(bboxs)
#         f.close()
#         img = np.array(img)
#         bbs = BoundingBoxesOnImage(rects,shape=img.shape)
#         count += 1
#         phase_count[phase] += 1
        
#         phase_count = rotate(img=img,bbs=bbs,specie="pf",offset=15,phase_count=phase_count)
        
#         aug_hor, bbs_hor, phase_count = flipHor(img=img,bbs=bbs,specie="pf",phase_count=phase_count)
#         phase_count = rotate(img=aug_hor,bbs=bbs_hor,specie="pf",offset=15,phase_count=phase_count)
        
#         aug_ver, bbs_ver, phase_count = flipVer(img=aug_hor,bbs=bbs_hor,specie="pf",phase_count=phase_count)
#         phase_count = rotate(img=aug_ver,bbs=bbs_ver,specie="pf",offset=15,phase_count=phase_count)
        
#         aug_ver2, bbs_ver2, phase_count = flipVer(img=img,bbs=bbs,specie="pf",phase_count=phase_count)
#         phase_count = rotate(img=aug_ver2,bbs=bbs_ver2,specie="pf",offset=15,phase_count=phase_count)

In [3]:
# print(phase_count["train"])

In [4]:
# get image from pf_01
# labels = os.path.join('pf_01.json')
count = 1
# degs = [45,90,135,180,225,270,315]
data = (requests.get('https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_01-copy.json')).json()
images = data["_via_img_metadata"]
for key in images:
    url = images[key]["filename"]
    leng = len(images[key]["regions"])
    if count > 700 and count <= 1032:
        name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
        print("{} {}".format(count,url))
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        img = np.array(img)
        img_green = img[:,:,1]
        ret, img_bw = cv2.threshold(img_green, 190, 255, cv2.THRESH_BINARY_INV)
        height, width = img_bw.shape
        disk = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7))
        img_bw = cv2.morphologyEx(img_bw, cv2.MORPH_OPEN, disk)
        (numLabels, labels, stats, centroids) = cv2.connectedComponentsWithStats(img_bw, connectivity=8)
        data["_via_img_metadata"][key]["regions"] = []
        for i in range(0, numLabels):
            x = stats[i, cv2.CC_STAT_LEFT]
            y = stats[i, cv2.CC_STAT_TOP]
            w = stats[i, cv2.CC_STAT_WIDTH]
            h = stats[i, cv2.CC_STAT_HEIGHT]
            area = stats[i, cv2.CC_STAT_AREA]
            if area >= 2000 and area <= 15000:
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 3)
                data["_via_img_metadata"][key]["regions"].append({
                    "shape_attributes": {
                        "name": "rect",
                        "x": int(x),
                        "y": int(y),
                        "width": int(w),
                        "height": int(h)
                    },
                    "region_attributes": {
                        "type": "0"
                    }
                })
        plt.imshow
        break
    count += 1

631 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i09j63.jpg
632 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i09j64.jpg
633 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i09j65.jpg
634 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i09j66.jpg
635 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i09j67.jpg
636 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i09j68.jpg
637 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i09j69.jpg
638 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i09j70.jpg
639 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i09

705 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i10j64.jpg
706 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i10j65.jpg
707 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i10j66.jpg
708 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i10j67.jpg
709 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i10j68.jpg
710 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i10j69.jpg
711 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i10j70.jpg
712 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i10j71.jpg
713 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i10

779 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i11j65.jpg
780 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i11j66.jpg
781 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i11j67.jpg
782 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i11j68.jpg
783 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i11j69.jpg
784 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i11j70.jpg
785 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i11j71.jpg
786 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i11j72.jpg
787 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i11

853 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j66.jpg
854 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j67.jpg
855 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j68.jpg
856 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j69.jpg
857 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j70.jpg
858 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j71.jpg
859 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j72.jpg
860 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i12j73.jpg
861 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i13

927 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i13j67.jpg
928 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i13j68.jpg
929 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i13j69.jpg
930 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i13j70.jpg
931 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i13j71.jpg
932 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i13j72.jpg
933 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i13j73.jpg
934 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14j01.jpg
935 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14

1001 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14j68.jpg
1002 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14j69.jpg
1003 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14j70.jpg
1004 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14j71.jpg
1005 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14j72.jpg
1006 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i14j73.jpg
1007 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i15j01.jpg
1008 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Extended_i15j02.jpg
1009 http://map.ku.ac.th/imglabel/via/datasets/pf_ring/01/Data20200115-01_Wholeslide_Default_Ext

In [5]:
with open("sample.json", "w") as outfile:
            json.dump(data, outfile)

In [6]:
# get image from pf_test
# labels = os.path.join('pf_test.json')

# count = 1

# phase = "test"

# data = (requests.get('https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_test.json')).json()
# images = data["_via_img_metadata"]
# for key in images:
#     url = images[key]["filename"]
#     leng = len(images[key]["regions"])
#     bboxs = ""
#     if leng > 0:
#         name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
#         print("{} {}".format(count,url))
#         print(name)
#         response = requests.get(url)
#         img = Image.open(BytesIO(response.content))
#         w, h = img.size
#         img.save(os.path.join('../data/custom/images',name+".jpg"))
#         f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
#         rects = []
#         for box in images[key]["regions"]:
#             b = box["shape_attributes"]
#             x1 = b["x"]/w
#             y1 = b["y"]/h
#             bw = b["width"]/w
#             bh = b["height"]/h
#             xc = x1 + (bw/2)
#             yc = y1 + (bh/2)
#             lbl = box["region_attributes"]["type"]
#             if lbl == "1":
#                 lbl = "1"
#                 bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
#                 rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
#             elif lbl == "0":
#                 lbl = "0"
#                 bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
#         bboxs = bboxs[1:]
#         f.write(bboxs)
#         f.close()
#         img = np.array(img)
#         bbs = BoundingBoxesOnImage(rects,shape=img.shape)
#         count += 1
#         phase_count[phase] += 1

In [7]:
# get image from pf_02 (Test set)
# labels = os.path.join('pf_02.json')

# count = 1

# phase = "test"
# degs = [45,90,135,180,225,270,315]

# data = (requests.get('https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_02.json')).json()
# images = data["_via_img_metadata"]
# for key in images:
#     url = images[key]["filename"]
#     leng = len(images[key]["regions"])
#     bboxs = ""
#     if leng > 0 and count <= 100:
#         name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
#         print("{} {}".format(count,url))
#         print(name)
#         response = requests.get(url)
#         img = Image.open(BytesIO(response.content))
#         w, h = img.size
#         img.save(os.path.join('../data/custom/images',name+".jpg"))
#         f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
#         for box in images[key]["regions"]:
#             b = box["shape_attributes"]
#             x1 = b["x"]/w
#             y1 = b["y"]/h
#             bw = b["width"]/w
#             bh = b["height"]/h
#             xc = x1 + (bw/2)
#             yc = y1 + (bh/2)
#             lbl = box["region_attributes"]["type"]
#             if lbl == "2":
#                 lbl = "1"
#                 bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
#             elif lbl == "0":
#                 lbl = "0"
#                 bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
#         bboxs = bboxs[1:]
#         f.write(bboxs)
#         f.close()
#         phase_count[phase] += 1
#     count += 1